<a href="https://colab.research.google.com/github/hyndhavamahesh345/LLM-Powered-Domain-Specific-QA-Agent/blob/main/Domain_QA_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM-Powered Domain-Specific QA Agent



In [ ]:
# 🩺 DOMAIN Q&A CHATBOT USING MEDQUAD DATASET (HEALTH DOMAIN)


# STEP 1: Install required libraries
!pip install -q sentence-transformers faiss-cpu transformers gradio datasets accelerate --upgrade

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline
import gradio as gr


In [ ]:
# STEP 2: Load the MedQuAD Dataset

try:
    df = pd.read_csv("medquad.csv")
except FileNotFoundError:
    print("Error: 'medquad.csv' not found. Please upload the dataset file.")
    # Exit or handle the error appropriately if the file is not found
    exit()

# Standardize column names
df = df.rename(columns={"Question": "question", "Answer": "answer"})
df["source"] = "NIH MedQuAD"

if "question" not in df.columns or "answer" not in df.columns:
    print("Error: 'medquad.csv' must contain 'Question' and 'Answer' columns.")
    exit()

df = df[["question", "answer", "source"]]

# Show first few rows
print("✅ Dataset loaded successfully!")
print(df.head())

✅ Dataset loaded successfully!
                                 question  \
0                What is (are) Glaucoma ?   
1                  What causes Glaucoma ?   
2     What are the symptoms of Glaucoma ?   
3  What are the treatments for Glaucoma ?   
4                What is (are) Glaucoma ?   

                                              answer       source  
0  Glaucoma is a group of diseases that can damag...  NIH MedQuAD  
1  Nearly 2.7 million people have glaucoma, a lea...  NIH MedQuAD  
2  Symptoms of Glaucoma  Glaucoma can develop in ...  NIH MedQuAD  
3  Although open-angle glaucoma cannot be cured, ...  NIH MedQuAD  
4  Glaucoma is a group of diseases that can damag...  NIH MedQuAD  


In [ ]:
# STEP 3: Build Sentence Embeddings and FAISS Index

print("\n🔄 Creating sentence embeddings...")

# Use MiniLM model for lightweight but accurate embeddings
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Encode all questions into embeddings
# Handle potential non-string data in 'question' column
questions = df["question"].astype(str).tolist()
question_embeddings = embedder.encode(questions, show_progress_bar=True)

# Build FAISS index for similarity search
index = faiss.IndexFlatL2(question_embeddings.shape[1])
index.add(np.array(question_embeddings))

print(f"✅ FAISS index built with {index.ntotal} medical Q&A entries.")


🔄 Creating sentence embeddings...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/513 [00:00<?, ?it/s]

✅ FAISS index built with 16412 medical Q&A entries.


In [ ]:
# STEP 4: Define Semantic Search Function

def semantic_search(query, top_k=3):
    """Find top_k semantically similar questions"""
    query_embedding = embedder.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            "question": df.iloc[idx]["question"],
            "answer": df.iloc[idx]["answer"],
            "source": df.iloc[idx]["source"],
            "distance": float(distances[0][i])
        })
    return results


In [ ]:
# STEP 5: Add QA Model for Contextual Answer Refinement

print("\n🤖 Loading QA model...")
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

def answer_query(query):
    """Retrieve best context and refine answer"""
    results = semantic_search(query, top_k=1)
    if not results:
        return {"original_answer": "No relevant information found.", "refined_answer": "No relevant information found.", "source": ""}

    best = results[0]
    context = str(best["answer"]) # Ensure context is a string
    # Check if context is not empty or just whitespace before passing to pipeline
    if not context.strip():
         return {"original_answer": best["answer"], "refined_answer": "Could not refine answer based on provided context.", "source": best["source"]}


    try:
        refined = qa_pipeline(question=query, context=context)
        refined_answer = refined["answer"]
    except Exception as e:
        print(f"Error during QA pipeline processing: {e}")
        refined_answer = "Could not refine answer." # Provide a fallback message

    return {
        "original_answer": best["answer"],
        "refined_answer": refined_answer,
        "source": best["source"]
    }

# Test example
print("\n🧠 Example test:")
test_query = "What are the early signs of diabetes?"
test_result = answer_query(test_query)
print(f"Query: {test_query}")
print(f"Original Answer: {test_result['original_answer']}")
print(f"Refined Answer: {test_result['refined_answer']}")
print(f"Source: {test_result['source']}")



🤖 Loading QA model...


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu



🧠 Example test:
Query: What are the early signs of diabetes?
Original Answer: Diabetes is often called a "silent" disease because it can cause serious complications even before you have symptoms. Symptoms can also be so mild that you dont notice them. An estimated 8 million people in the United States have type 2 diabetes and dont know it, according to 2012 estimates by the Centers for Disease Control and Prevention (CDC). Common Signs Some common symptoms of diabetes are: - being very thirsty  - frequent urination  - feeling very hungry or tired  - losing weight without trying  - having sores that heal slowly  - having dry, itchy skin  - loss of feeling or tingling in the feet  - having blurry eyesight. being very thirsty frequent urination feeling very hungry or tired losing weight without trying having sores that heal slowly having dry, itchy skin loss of feeling or tingling in the feet having blurry eyesight. Signs of type 1 diabetes usually develop over a short period of time. Th

In [ ]:
# STEP 6: Launch Gradio Chatbot Interface

def chat_interface(query):
    if not query.strip():
        return "❗ Please enter a question."
    result = answer_query(query)
    return f"**Answer:** {result['refined_answer']}\n\n**Source:** {result['source']}"

# Create Gradio Interface
gr.Interface(
    fn=chat_interface,
    inputs="text",
    outputs="markdown",
    title="🩺 MedQuAD Medical Q&A Chatbot",
    description="Ask any health-related question — the bot finds and refines answers using NIH MedQuAD dataset.",
    theme="default"
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f7b7cda1fdccd1a33a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
